# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [5]:
import mxnet as mx
import tensorflow as tf
import pkgutil, types, pprint
import numpy as np

def doc_dump(base, docs={}):
    try:
        for a in dir(eval(base)):
            if isinstance(eval(base).__dict__.get(a), types.FunctionType):
                doc = eval(base).__dict__.get(a).__doc__
                docs[base + '.' + a] = base + '.' + a + (doc if doc else '')
        for importer, modname, ispkg in pkgutil.walk_packages(path=eval(base).__path__,
                                                              prefix=base + '.',
                                                              onerror=lambda e: None):
            docs = doc_dump(modname, docs)
    except Exception as e:
        pass
    return docs
        
tf_docs = doc_dump('tf', {})
mx_docs = doc_dump('mx', {})

In [6]:
tf_labels, tf_values = zip(*list(tf_docs.items()))
mx_labels, mx_values = zip(*list(mx_docs.items()))

In [ ]:
HIDDEN_UNITS = 100
LAYERS = 1
BATCH_SIZE = 1
INPUT_SIZE = 1

net = mx.gluon.rnn.LSTM(HIDDEN_UNITS, LAYERS, input_size=1)
ctx = mx.cpu()
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

trainer = mx.gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})
l2loss = mx.gluon.loss.L2Loss()

for epoch in range(10):
    for i, x in enumerate(tf_values + mx_values):
        for j, y in enumerate(tf_values + mx_values):
            with mx.autograd.record():
                output_x = net(mx.nd.array([ord(xx) for xx in x]).reshape((-1, BATCH_SIZE, INPUT_SIZE)))
                output_y = net(mx.nd.array([ord(yy) for yy in y]).reshape((-1, BATCH_SIZE, INPUT_SIZE)))
                # Just compute loss on last output
                loss = (-1 if i == j else 1) * l2loss(output_x[len(x)-1, :, :], output_y[len(y)-1, :, :])
                loss.backward()
            trainer.step(BATCH_SIZE)
            print(mx.nd.mean(loss).asscalar())
        raise